In [1]:
# !pip install -q matplotlib

In [2]:
# TODO: Change this as required
CHUNK_NUM = "04"
prefix = "shard_"
input_f = prefix + CHUNK_NUM
output_dir = 'outputs/'

output_f = output_dir  + "output_" + prefix + CHUNK_NUM + '.csv'

In [3]:
import pandas as pd
import re

from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

import pickle
from typing import List

import os
import matplotlib.pyplot as plt

from tqdm.notebook import tqdm
tqdm.pandas()

In [4]:
df = pd.read_json(input_f, lines=True)

In [5]:
def clean_text(text):
    # Remove HTML tags
    text = re.sub(r'<.*?>', '', text)
    # Remove special characters like &gt;
    text = re.sub(r'&\w+;', '', text)
    return text.lstrip('1234567890')

In [6]:
import time
import pickle

cnt = 0
total = 0

start_time = time.time()

partial_rows = []

for i in range(df.shape[0]):
  for j in range(len(df['posts'][i])):
    total += 1
    if 'perspectives' not in df['posts'][i][j] or 'com' not in df['posts'][i][j] or 'no' not in df['posts'][i][j] or 'now' not in df['posts'][i][j]:
      continue

    ps = df['posts'][i][j]['perspectives']
    c = clean_text(df['posts'][i][j]['com'])
    no = df['posts'][i][j]['no']
    ts = df['posts'][i][j]['now']

    if len(c) < 15:
      continue

    if max(ps.values()) >= 0.5:
      continue

    if 'http' in c:
      continue

    transformed_row = {k: v for k, v in ps.items()}
    transformed_row['com'] = c
    transformed_row['no'] = no
    transformed_row['ts'] = ts

    partial_rows.append(transformed_row)
    cnt += 1
    # print(ps)
    # print(c)
    # print(no)
    # print(ts)

transformed_df = pd.DataFrame(partial_rows)
transformed_df.to_csv(input_f + '_transformed.csv', index_label = 'id')

#pickle.dump(partial_rows, open(input_f + '_transformed.pkl', 'wb'))
end_time = time.time()

print(cnt)
print(total)

secs = end_time - start_time
per_entry = secs / total
print(f"For 100k entries it should take: {100000 * per_entry}")
print(f"Filtered out {100.0 * (total - cnt) / total} % of entries")


72023
252423
For 100k entries it should take: 5.632334347178291
Filtered out 71.46733855472758 % of entries


In [7]:
final_df = pd.read_csv(input_f + '_transformed.csv')
final_df

,id,TOXICITY,SEVERE_TOXICITY,INFLAMMATORY,PROFANITY,INSULT,OBSCENE,SPAM,com,no,ts
0,0,0.046786,0.013702,0.162792,0.026061,0.034172,0.045205,0.022095,What&#039;s his name again?,81750325,07/19/16(Tue)09:59:57
1,1,0.071378,0.054332,0.303538,0.055032,0.065300,0.294497,0.249733,Alberto Barbosa,81751323,07/19/16(Tue)10:10:42
2,2,0.187903,0.086607,0.434657,0.089907,0.148738,0.248897,0.402654,his nameMy wife melanoma,81751649,07/19/16(Tue)10:14:04
3,3,0.282117,0.157774,0.391319,0.162552,0.234456,0.307138,0.027637,Those lips are made for slurping not speaking.,81752792,07/19/16(Tue)10:25:11
4,4,0.066193,0.011667,0.472857,0.026234,0.046301,0.078854,0.039636,increasingly nervous CTR shill millionth time ...,81710025,07/19/16(Tue)01:21:12
...,...,...,...,...,...,...,...,...,...,...,...
72018,72018,0.088789,0.075430,0.267800,0.069168,0.094825,0.178767,0.295806,Culturally enriched.,82044349,07/21/16(Thu)07:24:32
72019,72019,0.089674,0.066044,0.199570,0.099904,0.106668,0.488974,0.132810,context for an austard?,82044434,07/21/16(Thu)07:25:42
72020,72020,0.385499,0.255282,0.437159,0.221318,0.257255,0.107845,0.055849,tasing someone who has a gunWhat could go wrong?,82044603,07/21/16(Thu)07:28:32
72021,72021,0.368042,0.213973,0.490678,0.198197,0.336711,0.016719,0.062815,About the 13th Amendment? It abolished slavery...,82044747,07/21/16(Thu)07:30:38


In [8]:
# Load the model and tokenizer once
model_name = 'microsoft/DialoGPT-medium'
tokenizer = AutoTokenizer.from_pretrained(model_name)
# tokenizer.padding_side = "left"
# tokenizer.pad_token = '<pad>'
model = AutoModelForCausalLM.from_pretrained(model_name).to('cuda')

def get_dialogpt_response(comment):
    # Tokenize the input comment
    input_ids = tokenizer.encode(comment + tokenizer.eos_token, return_tensors="pt").to('cuda')

    # Generate a response
    response_ids = model.generate(input_ids, max_length=1000, pad_token_id=tokenizer.eos_token_id)
    response = tokenizer.decode(response_ids[:, input_ids.shape[-1]:][0], skip_special_tokens=True)

    return response

In [9]:
import pandas as pd

def process_and_save(final_df, func, b_chunk_size=200):
    """Applies a function to the DataFrame in chunks and saves periodically."""

    for i in range(0, len(final_df), b_chunk_size):
        chunk = final_df.iloc[i:i + b_chunk_size]
        chunk['response'] = chunk['com'].progress_apply(func)

        # Append to CSV file (mode='a' for append)
        chunk.to_csv(output_f, mode='a', header=i == 0, index=False)

process_and_save(final_df, get_dialogpt_response)

  0%|          | 0/200 [00:00<?, ?it/s]

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
/scratch/1028977/ipykernel_2155932/774280913.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chunk['response'] = chunk['com'].progress_apply(func)


  0%|          | 0/200 [00:00<?, ?it/s]

/scratch/1028977/ipykernel_2155932/774280913.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chunk['response'] = chunk['com'].progress_apply(func)


  0%|          | 0/200 [00:00<?, ?it/s]

...

/scratch/1028977/ipykernel_2155932/774280913.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chunk['response'] = chunk['com'].progress_apply(func)


  0%|          | 0/200 [00:00<?, ?it/s]

/scratch/1028977/ipykernel_2155932/774280913.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chunk['response'] = chunk['com'].progress_apply(func)


  0%|          | 0/200 [00:00<?, ?it/s]

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [10]:
sanity = pd.read_csv(output_f)
sanity

,id,TOXICITY,SEVERE_TOXICITY,INFLAMMATORY,PROFANITY,INSULT,OBSCENE,SPAM,com,no,ts,response
0,0,0.046786,0.013702,0.162792,0.026061,0.034172,0.045205,0.022095,What&#039;s his name again?,81750325,07/19/16(Tue)09:59:57,I think it's a reference to the movie The Inte...
1,1,0.071378,0.054332,0.303538,0.055032,0.065300,0.294497,0.249733,Alberto Barbosa,81751323,07/19/16(Tue)10:10:42,I think he's a good fit for the Clippers.
2,2,0.187903,0.086607,0.434657,0.089907,0.148738,0.248897,0.402654,his nameMy wife melanoma,81751649,07/19/16(Tue)10:14:04,His name is Robert Paulson.
3,3,0.282117,0.157774,0.391319,0.162552,0.234456,0.307138,0.027637,Those lips are made for slurping not speaking.,81752792,07/19/16(Tue)10:25:11,"I'm not sure if you're being sarcastic or not,..."
4,4,0.066193,0.011667,0.472857,0.026234,0.046301,0.078854,0.039636,increasingly nervous CTR shill millionth time ...,81710025,07/19/16(Tue)01:21:12,I'm not a shill. I'm a shill for the people wh...
...,...,...,...,...,...,...,...,...,...,...,...,...
72018,72018,0.088789,0.075430,0.267800,0.069168,0.094825,0.178767,0.295806,Culturally enriched.,82044349,07/21/16(Thu)07:24:32,"I'm not sure if you're being sarcastic or not,..."
72019,72019,0.089674,0.066044,0.199570,0.099904,0.106668,0.488974,0.132810,context for an austard?,82044434,07/21/16(Thu)07:25:42,I think it's a reference to the movie The Big ...
72020,72020,0.385499,0.255282,0.437159,0.221318,0.257255,0.107845,0.055849,tasing someone who has a gunWhat could go wrong?,82044603,07/21/16(Thu)07:28:32,"I'm not sure if you're serious or not, but I'm..."
72021,72021,0.368042,0.213973,0.490678,0.198197,0.336711,0.016719,0.062815,About the 13th Amendment? It abolished slavery...,82044747,07/21/16(Thu)07:30:38,I'm not sure what you're getting at.
